In [12]:
# =====================================================================
# Processo de Automatización NOKIA 4G
# =====================================================================

import os, glob, io, re
from pathlib import Path
import pandas as pd
from IPython.lib.deepreload import load_next
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font, Border, PatternFill
from openpyxl.utils import get_column_letter
import numpy as np
from datetime import date
import unicodedata
from charset_normalizer import from_path


# Ruta base (ajústala si cambia)
#BASE_DIR = Path(r"C:\Users\EAlor\OneDrive - ACS Solutions\Documents\AT&T\LST Cell Ran\Nokia New\Nokia Noviembre")
#BASE_DIR = Path(r"C:\Users\EAlor\OneDrive - ACS Solutions\Documents\AT&T\LST Cell Ran\Nokia New\XML_Output\Diciembre")
BASE_DIR = Path(r"C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Nokia\XML_Output\Febrero")

HEADERS = "AT&T_Site_Name", "Site ID", "VERSION", "DISTNAME", "MOID", "furtherPlmnIdL_mcc_mnc_autoRemovalAllowed_cellReserve", "mcc", "mnc", "name", "administrativeState", "cellName", "cellTechnology", "cellType", "eutraCelId", "moPrMappingList_lteNrDualConnectSupport_mcc_mnc_mncLength_moPrId", "nbIoTMode", "operationalState", "phyCellId", "tac", "actCatM", "dlChBw", "dlMimoMode", "earfcnDL", "earfcnUL", "maxNumUeDl", "maxNumUeUl", "rootSeqIndex", "ulChBw", "LAT", "LON", "MME TEF"


# Lista de encabezados, en el orden requerido
HEADER_LNCEL = ["VERSION", "DISTNAME", "MOID", "furtherPlmnIdL_mcc_mnc_autoRemovalAllowed_cellReserve", "mcc", "mnc", "name", "administrativeState", "cellName", "cellTechnology", "cellType", "eutraCelId", "moPrMappingList_lteNrDualConnectSupport_mcc_mnc_mncLength_moPrId", "nbIoTMode", "operationalState", "phyCellId", "tac"]

HEADER_MRBTS = ["FILENAME", "DATETIME", "VERSION", "DISTNAME", "MOID", "name", "altitude", "btsName", "latitude", "longitude", "blockingState"]

HEADER_LNCEL_FDD = ["VERSION", "DISTNAME", "MOID", "actCatM", "dlChBw", "dlMimoMode", "earfcnDL", "earfcnUL", "maxNumUeDl", "maxNumUeUl", "rootSeqIndex", "ulChBw"]

HEADER_LNMME = ["DISTNAME", "ipAddrPrim", "ipAddrSec"]


In [13]:
def read_csv_files(filename: str, header, encoder):
    filepath = str(BASE_DIR / f"{filename}")
    # Leer sólo las columnas necesarias del csv
    df = pd.read_csv(filepath, usecols = header, encoding = encoder, dtype=str)[header]


#    print(lncel_df.head())

    return df


In [14]:
lncel_df = read_csv_files("LNCEL.csv", HEADER_LNCEL, "utf-8")
mrbts_df = read_csv_files("MRBTS.csv", HEADER_MRBTS, "latin-1")
lncel_fdd_df = read_csv_files("LNCEL_FDD.csv", HEADER_LNCEL_FDD, "utf-8")
lnmme_df = read_csv_files("LNMME.csv",HEADER_LNMME, "utf-8")


print("Shape LNCEL original:", lncel_df.shape, "\nShape MRBTS original:", mrbts_df.shape, "\nShape LNCEL_FDD original:", lncel_fdd_df.shape, "\nShape LNMME original:", lnmme_df.shape)
# print(lncel_fdd_df.head(10).to_string(index=False))

try:
    display(lncel_df.head(5))
    display(mrbts_df.head(5))
    display(lncel_fdd_df.head(5))
    display(lnmme_df.head(5))
except NameError:
    # Por si no estás en notebook
    print(lncel_df.head(5).to_string(index=False))
    print(mrbts_df.head(5).to_string(index=False))
    print(lncel_fdd_df.head(5).to_string(index=False))
    print(lnmme_df.head(5).to_string(index=False))



Shape LNCEL original: (19000, 17) 
Shape MRBTS original: (1523, 11) 
Shape LNCEL_FDD original: (16843, 12) 
Shape LNMME original: (7517, 3)


,VERSION,DISTNAME,MOID,furtherPlmnIdL_mcc_mnc_autoRemovalAllowed_cellReserve,mcc,mnc,name,administrativeState,cellName,cellTechnology,cellType,eutraCelId,moPrMappingList_lteNrDualConnectSupport_mcc_mnc_mncLength_moPrId,nbIoTMode,operationalState,phyCellId,tac
0,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-21,106423232,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_S,unlocked,NMLGUASAL0477_1_S,FDD,large,28340501,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170
1,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-22,106423233,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_2_S,unlocked,NMLGUASAL0477_2_S,FDD,large,28340502,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,61,6170
2,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-23,106423235,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_3_S,unlocked,NMLGUASAL0477_3_S,FDD,large,28340503,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,62,6170
3,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-31,106423239,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_J,unlocked,NMLGUASAL0477_1_J,FDD,large,28340511,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170
4,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-32,106423267,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_2_J,unlocked,NMLGUASAL0477_2_J,FDD,large,28340512,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,61,6170


,FILENAME,DATETIME,VERSION,DISTNAME,MOID,name,altitude,btsName,latitude,longitude,blockingState
0,All_Nokia_20260126.xml,2026-01-26T12:20:19.000-06:00,SBTS22R2_2112_100,PLMN-PLMN/MRBTS-10001,9678559,NdB_RR_LAB_MER,2261,ToyCell_MERIDA,"+19Â°32'39.627""","-99Â°12'12.098""",NaN
1,All_Nokia_20260126.xml,2026-01-26T12:20:19.000-06:00,SBTS22R2_2112_100,PLMN-PLMN/MRBTS-10002,13335834,ToyCell-2,NaN,T-Veracruz,NaN,NaN,NaN
2,All_Nokia_20260126.xml,2026-01-26T12:20:19.000-06:00,SBTS24R1_2322_100,PLMN-PLMN/MRBTS-10004,64395764,10004-AGUAGU0025-ESTRELLA,NaN,10004-AGUAGU0025-ESTRELLA,NaN,NaN,Unblocked
3,All_Nokia_20260126.xml,2026-01-26T12:20:19.000-06:00,SBTS24R1_2322_100,PLMN-PLMN/MRBTS-10006,28730021,10006-AGUAGU0008-CAMPESTRE,1870,10006-AGUAGU0008-CAMPESTRE,"+21Â°55'34.323""","-102Â°19'04.979""",Unblocked
4,All_Nokia_20260126.xml,2026-01-26T12:20:19.000-06:00,SBTS24R1_2322_100,PLMN-PLMN/MRBTS-10007,57268286,10007-AGUAGU0048-VILLA TERESA,NaN,10007-AGUAGU0048-VILLA TERESA,NaN,NaN,Unblocked


,VERSION,DISTNAME,MOID,actCatM,dlChBw,dlMimoMode,earfcnDL,earfcnUL,maxNumUeDl,maxNumUeUl,rootSeqIndex,ulChBw
0,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-21/L...,106424134,false,10 MHz,Closed Loop Mimo,650,18650,14,14,600,10 MHz
1,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-22/L...,106424098,false,10 MHz,Closed Loop Mimo,650,18650,14,14,610,10 MHz
2,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-23/L...,106424033,false,10 MHz,Closed Loop Mimo,650,18650,14,14,620,10 MHz
3,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-31/L...,106424560,true,15 MHz,Closed Loop MIMO (4x4),2225,20225,14,14,600,15 MHz
4,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-32/L...,106424468,true,15 MHz,Closed Loop MIMO (4x4),2225,20225,14,14,610,15 MHz


,DISTNAME,ipAddrPrim,ipAddrSec
0,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNMME-0,10.216.60.34,10.216.60.35
1,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNMME-1,10.38.147.16,10.38.147.17
2,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNMME-2,10.35.180.40,10.35.180.41
3,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNMME-22,201.162.170.67,NaN
4,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNMME-25,201.162.219.14,NaN


In [15]:
# =====================================================================
# 1) Insertar columnas AT&TSite_Name y Site ID
# =====================================================================

# Copia para no tocar los originales
lncel_df_mod = lncel_df.copy()


print("Shape original:", lncel_df_mod.shape)

# Crear una clave en df con la concatenación compleja
substr_distname = lncel_df_mod['DISTNAME'].str.split('/', n=2).str[:2].str.join('/')

# --- Crear columna 'AT&T_Site_Name' a partir de mrbts_df ---
# 1. Crear un mapeo DISTNAME -> name
mapa = dict(zip(mrbts_df['DISTNAME'], mrbts_df['name']))

# 2. Generar la clave intermedia a partir de DISTNAME
substr_distname = lncel_df_mod['DISTNAME'].str.split('/', n=2).str[:2].str.join('/')

# 3. Mapear con el diccionario
tmp_name = substr_distname.map(mapa)

# 4. Aplicar la lógica condicional:
#    - Si contiene '-', usar la segunda parte (x.split('-', 2)[1])
#    - Si no, dejar el valor original
#    - Si está vacío, dejar NaN
lncel_df_mod.insert(
    0,
    'AT&T_Site_Name',
    tmp_name.apply(
        lambda x: x.split('-', 2)[1].strip() if isinstance(x, str) and '-' in x else x
    )
)

# 5. (Opcional) Rellenar los NaN con el valor original de mrbts_df['name']
lncel_df_mod['AT&T_Site_Name'].fillna(substr_distname.map(mapa), inplace=True)


"""
# Crear un diccionario desde mrbts_df: DISTNAME -> name
mapa = dict(zip(mrbts_df['DISTNAME'], mrbts_df['name']))

# Aplicar el mapeo
tmp_name = substr_distname.map(mapa)

# Si hay '-', usar la segunda parte; si no, dejar el nombre original
lncel_df_mod.insert(
    0,
    'AT&T_Site_Name',
    tmp_name.apply(lambda x: x.split('-', 2)[1] if isinstance(x, str) and '-' in x else x)
)

# Insertar la nueva columna AT&T_Site_Name en posición 0 con el valor mapeado
# lncel_df_mod.insert(0, 'AT&T_Site_Name', substr_distname.map(mapa).str.split('-', n=2).str[1])
"""
# Mostrar solo las filas donde AT&T_Site_Name es NaN
print("Mostrar los que quedaron nulos")
print(lncel_df_mod[lncel_df_mod["AT&T_Site_Name"].isna()][["DISTNAME", "AT&T_Site_Name"]])

# Insertar la nueva columna Site ID extrayendo el texto deseado de DISTNAME
lncel_df_mod.insert(1, 'Site ID', lncel_df_mod['DISTNAME'].str.split('-', n=3).str[3].str.split('/',n=2).str[0] )

print("Shape nuevo:", lncel_df_mod.shape)

# Vista de verificación (muestra solo unas filas)

pd.set_option("display.max_columns", None)  # opcional
try:
    display(lncel_df_mod.head(34))
except NameError:
    # Por si no estás en notebook
    print(lncel_df_mod.head(5).to_string(index=False))


Shape original: (19000, 17)
Mostrar los que quedaron nulos
                                           DISTNAME AT&T_Site_Name
192    PLMN-PLMN/MRBTS-110690/LNBTS-110690/LNCEL-21            NaN
193    PLMN-PLMN/MRBTS-110690/LNBTS-110690/LNCEL-22            NaN
194    PLMN-PLMN/MRBTS-110690/LNBTS-110690/LNCEL-23            NaN
195    PLMN-PLMN/MRBTS-110690/LNBTS-110690/LNCEL-31            NaN
196    PLMN-PLMN/MRBTS-110690/LNBTS-110690/LNCEL-32            NaN
197    PLMN-PLMN/MRBTS-110690/LNBTS-110690/LNCEL-33            NaN
198    PLMN-PLMN/MRBTS-110690/LNBTS-110690/LNCEL-34            NaN
199    PLMN-PLMN/MRBTS-110690/LNBTS-110690/LNCEL-35            NaN
200    PLMN-PLMN/MRBTS-110690/LNBTS-110690/LNCEL-36            NaN
201    PLMN-PLMN/MRBTS-110690/LNBTS-110690/LNCEL-41            NaN
202    PLMN-PLMN/MRBTS-110690/LNBTS-110690/LNCEL-42            NaN
203    PLMN-PLMN/MRBTS-110690/LNBTS-110690/LNCEL-43            NaN
204    PLMN-PLMN/MRBTS-110690/LNBTS-110690/LNCEL-51            NaN
205

C:\Users\EAlor\AppData\Local\Temp\ipykernel_6092\152661992.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  lncel_df_mod['AT&T_Site_Name'].fillna(substr_distname.map(mapa), inplace=True)


,AT&T_Site_Name,Site ID,VERSION,DISTNAME,MOID,furtherPlmnIdL_mcc_mnc_autoRemovalAllowed_cellReserve,mcc,mnc,name,administrativeState,cellName,cellTechnology,cellType,eutraCelId,moPrMappingList_lteNrDualConnectSupport_mcc_mnc_mncLength_moPrId,nbIoTMode,operationalState,phyCellId,tac
0,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-21,106423232,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_S,unlocked,NMLGUASAL0477_1_S,FDD,large,28340501,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170
1,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-22,106423233,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_2_S,unlocked,NMLGUASAL0477_2_S,FDD,large,28340502,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,61,6170
2,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-23,106423235,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_3_S,unlocked,NMLGUASAL0477_3_S,FDD,large,28340503,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,62,6170
3,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-31,106423239,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_J,unlocked,NMLGUASAL0477_1_J,FDD,large,28340511,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170
4,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-32,106423267,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_2_J,unlocked,NMLGUASAL0477_2_J,FDD,large,28340512,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,61,6170
5,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-33,106423317,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_3_J,unlocked,NMLGUASAL0477_3_J,FDD,large,28340513,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,62,6170
6,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-34,106423406,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_K,unlocked,NMLGUASAL0477_1_K,FDD,large,28340514,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170
7,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-35,106423661,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_2_K,unlocked,NMLGUASAL0477_2_K,FDD,large,28340515,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,61,6170
8,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-36,106423663,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_3_K,unlocked,NMLGUASAL0477_3_K,FDD,large,28340516,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,62,6170
9,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-41,106423534,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_T,unlocked,NMLGUASAL0477_1_T,FDD,large,28340521,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170


In [16]:
# =====================================================================
# 2) Información de LNCEL_FDD
# =====================================================================

lncel_df_merged = lncel_df_mod.copy()

print("Shape original:", lncel_df_merged.shape)

# Remueve la ultima parte de DISTNAME en LNCEL_FDD para que haga match con DISTNAME de LNCEL
lncel_fdd_df['DISTNAME'] = lncel_fdd_df['DISTNAME'].str.rsplit('/', n=1).str[0]

columnas_a_insertar = ["actCatM", "dlChBw", "dlMimoMode", "earfcnDL", "earfcnUL", "maxNumUeDl", "maxNumUeUl", "rootSeqIndex", "ulChBw"]

# limpia string de DISTNAME
lncel_df_merged["DISTNAME"] = lncel_df_merged["DISTNAME"].astype(str).str.strip().str.upper()
lncel_fdd_df["DISTNAME"] = lncel_fdd_df["DISTNAME"].astype(str).str.strip().str.upper()

coincidencias = set(lncel_df_merged["DISTNAME"]) & set(lncel_fdd_df["DISTNAME"])
print(len(coincidencias))

lncel_df_merged = lncel_df_merged.merge(
    lncel_fdd_df[["DISTNAME"] + columnas_a_insertar],
    on = "DISTNAME",
    how = "left")

print("Shape nuevo:", lncel_df_merged.shape)

# Vista de verificación (muestra solo unas filas)
pd.set_option("display.max_columns", None)  # opcional
try:
    display(lncel_df_merged.head(5))
except NameError:
    # Por si no estás en notebook
    print(lncel_df_merged.head(5).to_string(index=False))


Shape original: (19000, 19)
16843
Shape nuevo: (19000, 28)


,AT&T_Site_Name,Site ID,VERSION,DISTNAME,MOID,furtherPlmnIdL_mcc_mnc_autoRemovalAllowed_cellReserve,mcc,mnc,name,administrativeState,cellName,cellTechnology,cellType,eutraCelId,moPrMappingList_lteNrDualConnectSupport_mcc_mnc_mncLength_moPrId,nbIoTMode,operationalState,phyCellId,tac,actCatM,dlChBw,dlMimoMode,earfcnDL,earfcnUL,maxNumUeDl,maxNumUeUl,rootSeqIndex,ulChBw
0,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-21,106423232,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_S,unlocked,NMLGUASAL0477_1_S,FDD,large,28340501,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170,false,10 MHz,Closed Loop Mimo,650,18650,14,14,600,10 MHz
1,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-22,106423233,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_2_S,unlocked,NMLGUASAL0477_2_S,FDD,large,28340502,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,61,6170,false,10 MHz,Closed Loop Mimo,650,18650,14,14,610,10 MHz
2,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-23,106423235,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_3_S,unlocked,NMLGUASAL0477_3_S,FDD,large,28340503,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,62,6170,false,10 MHz,Closed Loop Mimo,650,18650,14,14,620,10 MHz
3,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-31,106423239,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_J,unlocked,NMLGUASAL0477_1_J,FDD,large,28340511,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170,true,15 MHz,Closed Loop MIMO (4x4),2225,20225,14,14,600,15 MHz
4,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-32,106423267,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_2_J,unlocked,NMLGUASAL0477_2_J,FDD,large,28340512,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,61,6170,true,15 MHz,Closed Loop MIMO (4x4),2225,20225,14,14,610,15 MHz


In [17]:
# def _is_blank(s: pd.Series) -> pd.Series:
#    return s.isna() | s.astype(str).str.strip().eq("")

# =====================================================================
# 3) LAT/LON desde All_Nokia_4G_{YYYYMM} (mes anterior)
# =====================================================================

today = date.today()
prev_year  = today.year if today.month > 1 else today.year - 1
prev_month = today.month - 1 or 12
yyyymm = f"{prev_year}{prev_month:02d}"
print(yyyymm)

# an_path = BASE_DIR / f"All_Nokia_4G_{yyyymm}.xlsx"
an_path = BASE_DIR / f"All_Nokia_4G_20260121.xlsx"
an_df = pd.read_excel(an_path, usecols=["AT&T_Site_Name", "LAT", "LON"])
an_df["AT&T_Site_Name"] = an_df["AT&T_Site_Name"].astype(str).str.strip()
an_df = an_df.drop_duplicates(subset=["AT&T_Site_Name"], keep="first")
display(an_df.head(5))

lncel_df_merged["AT&T_Site_Name"] = lncel_df_merged["AT&T_Site_Name"].astype(str).str.strip()

merged_df = lncel_df_merged.merge(
    an_df,
    on="AT&T_Site_Name",
    how="left",
    suffixes=("", "_an")
)


print("Shape nuevo:", merged_df.shape)

# Vista de verificación (muestra solo unas filas)
pd.set_option("display.max_columns", None)  # opcional
try:
    display(merged_df.head(5))
except NameError:
    # Por si no estás en notebook
    print(merged_df.head(10).to_string(index=False))


202512


,AT&T_Site_Name,LAT,LON
0,GUASAL0477,20.560011,-101.168181
21,GUACEL1699,20.511615,-100.795872
42,GUAABS0123,20.449425,-101.534857
48,GUASMA3326,20.851917,-100.506032
54,GUALEO0421,21.116713,-101.595124


Shape nuevo: (19000, 30)


,AT&T_Site_Name,Site ID,VERSION,DISTNAME,MOID,furtherPlmnIdL_mcc_mnc_autoRemovalAllowed_cellReserve,mcc,mnc,name,administrativeState,cellName,cellTechnology,cellType,eutraCelId,moPrMappingList_lteNrDualConnectSupport_mcc_mnc_mncLength_moPrId,nbIoTMode,operationalState,phyCellId,tac,actCatM,dlChBw,dlMimoMode,earfcnDL,earfcnUL,maxNumUeDl,maxNumUeUl,rootSeqIndex,ulChBw,LAT,LON
0,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-21,106423232,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_S,unlocked,NMLGUASAL0477_1_S,FDD,large,28340501,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170,false,10 MHz,Closed Loop Mimo,650,18650,14,14,600,10 MHz,20.560011,-101.168181
1,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-22,106423233,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_2_S,unlocked,NMLGUASAL0477_2_S,FDD,large,28340502,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,61,6170,false,10 MHz,Closed Loop Mimo,650,18650,14,14,610,10 MHz,20.560011,-101.168181
2,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-23,106423235,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_3_S,unlocked,NMLGUASAL0477_3_S,FDD,large,28340503,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,62,6170,false,10 MHz,Closed Loop Mimo,650,18650,14,14,620,10 MHz,20.560011,-101.168181
3,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-31,106423239,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_J,unlocked,NMLGUASAL0477_1_J,FDD,large,28340511,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170,true,15 MHz,Closed Loop MIMO (4x4),2225,20225,14,14,600,15 MHz,20.560011,-101.168181
4,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-32,106423267,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_2_J,unlocked,NMLGUASAL0477_2_J,FDD,large,28340512,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,61,6170,true,15 MHz,Closed Loop MIMO (4x4),2225,20225,14,14,610,15 MHz,20.560011,-101.168181


In [ ]:
### Información del EPT ###
if merged_df[["LAT", "LON"]].isna().any(axis=1).any():
    # Ejecuta el proceso si hay al menos un NaN en LAT o LON

    nan_count = merged_df[["LAT", "LON"]].isna().sum().sum()
    print("Se necesita buscar LAT/LON restantes en EPT")

    ruta_ept = BASE_DIR

    # Prefijo del archivo
    prefijo_ept = "EPT_ATT_UMTS_LTE_"

    # Busca archivo que empiece con el prefijo
    archivo = glob.glob(os.path.join(ruta_ept, f"{prefijo_ept}*.xlsx"))

    # Verifica si se encontró archivo
    if archivo:
        archivo_encontrado = archivo[0]
        nombre_archivo = os.path.basename(archivo_encontrado)

        # Lista de hojas a leer
        hojas_fijas = [
            "EPT_3G_LTE_OUTDOOR",
            "PLAN_OUTDOOR",
            "EPT_3G_LTE_INDOOR",
            "PLAN_INDOOR",
            "Eventos_Especiales"
        ]

        # Detecta automáticamente las hojas que contienen "Nokia" (para este vendor en particular)
        todas_las_hojas = pd.ExcelFile(archivo_encontrado, engine="openpyxl").sheet_names
        hojas_vendor = [h for h in todas_las_hojas if "nokia" in h.lower()]
        # Combina ambas listas (sin duplicar)
        hojas = list(dict.fromkeys(hojas_fijas + hojas_vendor))
        print(hojas)

        # Lee todas las hojas y agrega el nombre de la hoja en columna
        dfs = [
            pd.read_excel(archivo_encontrado, sheet_name=hoja, usecols=["AT&T_Site_Name", "Latitud", "Longitud"], engine="openpyxl")
            .assign(Hoja=hoja, Origen=nombre_archivo)
            for hoja in hojas
        ]

        # Concatena todo en un solo DataFrame
        df_EPT_inicial = pd.concat(dfs, ignore_index=True).drop_duplicates(subset=["AT&T_Site_Name"])

        # Unir con df principal (solo para los NaN)
        merged_df = merged_df.merge(df_EPT_inicial, on="AT&T_Site_Name", how="left", suffixes=("", "_extra"))
        merged_df["LAT"] = merged_df["LAT"].fillna(merged_df["Latitud"])
        merged_df["LON"] = merged_df["LON"].fillna(merged_df["Longitud"])
        merged_df = merged_df.drop(columns=["Latitud", "Longitud"])

        # Eliminar columnas auxiliares si existen
        merged_df = merged_df.drop(columns=[c for c in ["Hoja", "Origen"] if c in merged_df.columns])



Se necesita buscar LAT/LON restantes en EPT
['EPT_3G_LTE_OUTDOOR', 'PLAN_OUTDOOR', 'EPT_3G_LTE_INDOOR', 'PLAN_INDOOR', 'Eventos_Especiales', 'Overlay Nokia', 'Nokia_DSS', 'R&R Nokia']


In [10]:
# =====================================================================
# 4) Información IP
# =====================================================================

lnmme_df_mod = lnmme_df.copy()
out_df = merged_df.copy()

tef_path = BASE_DIR / f"IP TEF.xlsx"
ip_tef_df = pd.read_excel(tef_path, engine="openpyxl")
print("Shape original:", lnmme_df.shape)

try:
    display(lnmme_df.head(5))
except NameError:
    # Por si no estás en notebook
    print(lnmme_df_mod.head(5).to_string(index=False))

lnmme_df_mod["Vmme"] = lnmme_df_mod["ipAddrPrim"].map(ip_tef_df.set_index("IP Address")["Hostname"]).fillna("#N/A")

# Eliminar filas donde Vmme == "#N/A"
lnmme_df_mod = lnmme_df_mod[lnmme_df_mod["Vmme"] != "#N/A"].copy()

# Insertar nueva columna al inicio con el valor extraído
lnmme_df_mod.insert(
0,
"Site ID", # nombre de la nueva columna
lnmme_df_mod["DISTNAME"].str.extract(r"MRBTS-(\d+)")[0]
)

# Contar cuántos valores distintos de Vmme tiene cada Site ID
vmme_counts = (
    lnmme_df_mod.groupby("Site ID")["Vmme"]
    .nunique() # cuenta valores únicos
    .rename("MME TEF")
)


# Agregar esa información al final de lncel_df
out_df = out_df.merge(vmme_counts, on="Site ID", how="left")

if "MME TEF" in out_df.columns:
    out_df["MME TEF"] = out_df["MME TEF"].fillna(0).astype(int)
else:
    out_df["MME TEF"] = 0


# 3. Llenar con 0 los Site ID que no tengan coincidencia
out_df["MME TEF"] = out_df["MME TEF"].fillna(0).astype(int)



print("Shape nuevo:", lnmme_df.shape)

try:
    display(lnmme_df_mod.head(5))
    display(out_df.head(5))
except NameError:
    # Por si no estás en notebook
    print(lnmme_df_mod.head(5).to_string(index=False))
    print(out_df.head(5).to_string(index=False))



Shape original: (7517, 3)


,DISTNAME,ipAddrPrim,ipAddrSec
0,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNMME-0,10.216.60.34,10.216.60.35
1,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNMME-1,10.38.147.16,10.38.147.17
2,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNMME-2,10.35.180.40,10.35.180.41
3,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNMME-22,201.162.170.67,NaN
4,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNMME-25,201.162.219.14,NaN


Shape nuevo: (7517, 3)


,Site ID,DISTNAME,ipAddrPrim,ipAddrSec,Vmme
3,110705,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNMME-22,201.162.170.67,NaN,MTYCO4N
4,110705,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNMME-25,201.162.219.14,NaN,DFN1M4N
8,110724,PLMN-PLMN/MRBTS-110724/LNBTS-110724/LNMME-22,201.162.170.67,NaN,MTYCO4N
9,110724,PLMN-PLMN/MRBTS-110724/LNBTS-110724/LNMME-25,201.162.219.14,NaN,DFN1M4N
13,110521,PLMN-PLMN/MRBTS-110521/LNBTS-110521/LNMME-22,201.162.170.67,NaN,MTYCO4N


,AT&T_Site_Name,Site ID,VERSION,DISTNAME,MOID,furtherPlmnIdL_mcc_mnc_autoRemovalAllowed_cellReserve,mcc,mnc,name,administrativeState,cellName,cellTechnology,cellType,eutraCelId,moPrMappingList_lteNrDualConnectSupport_mcc_mnc_mncLength_moPrId,nbIoTMode,operationalState,phyCellId,tac,actCatM,dlChBw,dlMimoMode,earfcnDL,earfcnUL,maxNumUeDl,maxNumUeUl,rootSeqIndex,ulChBw,LAT,LON,MME TEF
0,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-21,106423232,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_S,unlocked,NMLGUASAL0477_1_S,FDD,large,28340501,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170,false,10 MHz,Closed Loop Mimo,650,18650,14,14,600,10 MHz,20.560011,-101.168181,2
1,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-22,106423233,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_2_S,unlocked,NMLGUASAL0477_2_S,FDD,large,28340502,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,61,6170,false,10 MHz,Closed Loop Mimo,650,18650,14,14,610,10 MHz,20.560011,-101.168181,2
2,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-23,106423235,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_3_S,unlocked,NMLGUASAL0477_3_S,FDD,large,28340503,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,62,6170,false,10 MHz,Closed Loop Mimo,650,18650,14,14,620,10 MHz,20.560011,-101.168181,2
3,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-31,106423239,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_J,unlocked,NMLGUASAL0477_1_J,FDD,large,28340511,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170,true,15 MHz,Closed Loop MIMO (4x4),2225,20225,14,14,600,15 MHz,20.560011,-101.168181,2
4,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-32,106423267,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_2_J,unlocked,NMLGUASAL0477_2_J,FDD,large,28340512,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,61,6170,true,15 MHz,Closed Loop MIMO (4x4),2225,20225,14,14,610,15 MHz,20.560011,-101.168181,2


In [11]:
# =====================================================================
# 7) Archivo final en excel
# =====================================================================

# === 0) Config y fecha actual ===
today   = date.today()
yyyymm  = f"{today.year}{today.month:02d}{today.day:02d}"

final_excel = BASE_DIR / f"All_Nokia_4G_{yyyymm}.xlsx"
tmp_excel   = BASE_DIR / f"~tmp_All_Nokia_4G_{yyyymm}.xlsx"

# Usa tu DataFrame final en memoria
df_out = out_df.copy()  # o df_sorted si ya lo traes ordenado

pd.set_option("display.max_columns", None)  # opcional
try:
    display(df_out.head(5))
except NameError:
    # Por si no estás en notebook
    print(df_out.head(5).to_string(index=False))

"""
# === 1) Asegurar columnas y orden base ===
HEADERS_FINAL = HEADERS[:]

for col in HEADERS_FINAL:
    if col not in df_out.columns:
        df_out[col] = pd.NA

df_out = df_out[HEADERS_FINAL]
"""

# === 2) Guardar sin formato, sin 'nan' ===
df_out.to_excel(final_excel, index=False, na_rep="")

# 2) Reabrir el MISMO archivo y aplicar formato
wb = load_workbook(final_excel)
ws = wb.active

ws.freeze_panes = "A2"
for col_idx, header in enumerate(HEADERS, start=1):
    cell = ws.cell(row=1, column=col_idx)
    cell.value = header
    cell.font = Font(name="Aptos Narrow", size=11)
    cell.alignment = Alignment(textRotation=90, horizontal="center", vertical="center", wrap_text=True)
    cell.border = Border()

# Agregar filtro automático en el header
ws.auto_filter.ref = ws.dimensions  # aplica el filtro a todo el rango con datos

wb.save(final_excel)
wb.close()

print(f"✅ Archivo final sin formato guardado → {final_excel}")

,AT&T_Site_Name,Site ID,VERSION,DISTNAME,MOID,furtherPlmnIdL_mcc_mnc_autoRemovalAllowed_cellReserve,mcc,mnc,name,administrativeState,cellName,cellTechnology,cellType,eutraCelId,moPrMappingList_lteNrDualConnectSupport_mcc_mnc_mncLength_moPrId,nbIoTMode,operationalState,phyCellId,tac,actCatM,dlChBw,dlMimoMode,earfcnDL,earfcnUL,maxNumUeDl,maxNumUeUl,rootSeqIndex,ulChBw,LAT,LON,MME TEF
0,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-21,106423232,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_S,unlocked,NMLGUASAL0477_1_S,FDD,large,28340501,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170,false,10 MHz,Closed Loop Mimo,650,18650,14,14,600,10 MHz,20.560011,-101.168181,2
1,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-22,106423233,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_2_S,unlocked,NMLGUASAL0477_2_S,FDD,large,28340502,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,61,6170,false,10 MHz,Closed Loop Mimo,650,18650,14,14,610,10 MHz,20.560011,-101.168181,2
2,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-23,106423235,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_3_S,unlocked,NMLGUASAL0477_3_S,FDD,large,28340503,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,62,6170,false,10 MHz,Closed Loop Mimo,650,18650,14,14,620,10 MHz,20.560011,-101.168181,2
3,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-31,106423239,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_1_J,unlocked,NMLGUASAL0477_1_J,FDD,large,28340511,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,60,6170,true,15 MHz,Closed Loop MIMO (4x4),2225,20225,14,14,600,15 MHz,20.560011,-101.168181,2
4,GUASAL0477,110705,xL24R1_2322_110,PLMN-PLMN/MRBTS-110705/LNBTS-110705/LNCEL-32,106423267,334&090&false&Not Reserved;334&03&false&Not Re...,334,050,NMLGUASAL0477_2_J,unlocked,NMLGUASAL0477_2_J,FDD,large,28340512,all&334&50&3&1;all&334&90&3&1;all&334&3&2&1,disabled,enabled,61,6170,true,15 MHz,Closed Loop MIMO (4x4),2225,20225,14,14,610,15 MHz,20.560011,-101.168181,2


✅ Archivo final sin formato guardado → C:\Users\EAlor\OneDrive - ACS Solutions\Documents\AT&T\LST Cell Ran\Nokia New\XML_Output\Enero\All_Nokia_4G_20260128.xlsx
